In [ ]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd

**You will need `fastplotlib` installed for the visualizations**

In [ ]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [ ]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

**Even on windows just use `/`, you do not have to worry about the annoying issue of `\\` and `\` on windows if you use `pathlib` :D**

In [ ]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [ ]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-cnmfe/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [ ]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty with the appropriate columns for mesmerize

In [ ]:
df

In [ ]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/data_endoscope.tif")

In [ ]:
params =\
{
    "main":
    {
        "gSig_filt": (3, 3),
        "pw_rigid": True,
        "max_shifts": (5, 5),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

In [ ]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params,
    item_name=movie_path.stem
)

df

In [ ]:
row = df.iloc[0]

In [ ]:
row.caiman.run()

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
df

In [ ]:
import tifffile

In [ ]:
input_movie = df.iloc[0].caiman.get_input_movie(tifffile.imread)
mcorr_movie = df.iloc[0].mcorr.get_output()

In [ ]:
from scipy.ndimage import convolve

In [ ]:
kernel = np.array([[-1, -1, -1, -1, -1],
                   [-1,  1,  2,  1, -1],
                   [-1,  2,  5,  2, -1],
                   [-1,  1,  2,  1, -1],
                   [-1, -1, -1, -1, -1]])

f = lambda frame: convolve(frame, kernel)

In [ ]:
iw = ImageWidget(
    [input_movie, mcorr_movie, input_movie, mcorr_movie], 
    vmin_vmax_sliders=True, 
    cmap="gnuplot2",
    frame_apply={0: lambda x: x, 1: lambda x: x, 2: f, 3:f}
)

iw.show()

In [ ]:
iw.window_funcs = {"t": (np.mean, 17)}

In [ ]:
corr = df.iloc[0].caiman.get_corr_image()
pnr = df.iloc[0].caiman.get_pnr_image()

iw = ImageWidget([corr, pnr], vmin_vmax_sliders=True, cmap="jet")
iw.show()

In [ ]:
params_cnmf =\
{
    "main":
    {
        'method_init': 'corr_pnr',  # use this for 1 photon
        'K': None,
        'gSig': (3, 3),
        'gSiz': (13, 13),
        'merge_thr': 0.7,
        'p': 1,
        'tsub': 2,
        'ssub': 1,
        'rf': 40,
        'stride': 20,
        'only_init': True,    # set it to True to run CNMF-E
        'nb': 0,
        'nb_patch': 0,
        'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
        'low_rank_background': None,
        'update_background_components': True,  # sometimes setting to False improve the results
        'min_corr': 0.8,
        'min_pnr': 10,
        'normalize_init': False,               # just leave as is
        'center_psf': True,                    # leave as is for 1 photon
        'ssub_B': 2,
        'ring_size_factor': 1.4,
        'del_duplicates': True,                # whether to remove duplicates from initialization
    }
}